In [134]:
# Renameios
#!/usr/bin/python
#
# Renaming iOS devices and push install app through MDM
#
# Single threading version 0.1
#
# Usage:
#
# Author:Kelvin J. July,2017
#
#
import pip 
import csv,sys,json,os,re
import urllib 
import time

try:
    from robobrowser import RoboBrowser
except :
    pip.main(['install', 'robobrowser'])
from StringIO import StringIO
import getpass,time
#import threading

try:
    from enum import Enum
except :
    pip.main(['install', 'enum'])
try:
    import lxml
except :
    pip.main(['install', 'lxml'])

try:
    from bs4 import BeautifulSoup
except :
    pip.main(['install', 'bs4'])

    
    
class Roles(Enum):
    Elementary = '18536'
    Secondary = '18985'
    TOP_LEVEL= '18535'
     
class OUs(Enum):
    ios = 'ios'
    tv = 'tv'
    ioslease= 'ioslease'
    elementary='1111' 
    secondary='2222'


class idevice(object):
    def __init__(self):
        self.attrs={'DeviceOwnership':None,
            'UserName':None,
                'EmailAddress':None,
                'Platform':None,
                'FriendlyName':None,
                'OSVersion':None,
                'DisplayModel':None ,
                'EnrollmentStatusName':None,
                'ComplianceStatusName':None ,
                'DisplayName':None,
                'ManagedBy':None,
                'AssetNumber':None,
                'SerialNumber':None,
                'LocationGroupName':None,
                'IsCompromisedYN': None,
                'WiFiIPAddress':None,
                'WiFiMacAddress':None,
                'Notes':None,
                'WnsStatus' :None,
                'DeviceId':None}

    def getAttrs(self):
		return self.attrs

def searchAll(browser,querystr):
    devices={}
    text=querystr
    text=urllib.quote_plus(text)
    
    queryurl='https://awmdm.com/AirWatch/Device/List/Search?Page=&Size=100&SearchText='+text+'ipads+ipad&CurrentView=Custom&Order=Descending&Sort=LastPingDate'

 
    #print queryurl
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Referer']='https://cnxxx.awmdm.com/AirWatch/'

  
    browser.open(queryurl)
 
    
    try:          
        result_trs= browser.find("div",{"id": "DeviceGrid"}).findAll('tr',{'data-action-names':"QueryAll LockDevice SendMessage ChangeOrganizationGroup DeleteDevice"})
        if len(result_trs)>0:
            print "Number of devices found: "+str(len(result_trs)) 
        else:
            print "Number of devices found: 0"
        for r in result_trs: 
            dev_id=None
            dev_sn=None
            tr=BeautifulSoup(str(r),'lxml')
            tags=tr.findAll('td')
            for t in tags:      
                if t.has_attr('data-property') and t.attrs['data-property']== 'SerialNumber':
                    dev_sn=t.text

            tags=tr.findAll('input')
            for t in tags:
                if t.attrs['type']=='checkbox':
                    #print t.attrs['value']
                    dev_id=t.attrs['value']
            devices[dev_sn]=dev_id 
    except:
        print 'Error on finding serial number...'
        #sys.exit(1)
    
    #src = str(browser.parsed())

    #result = re.search('%s(.*)%s' % (start, end), src).group(1)
    #print devices  
    with open('sns.json', 'w') as file:
         file.write(json.dumps(devices))
    return devices

def queryOne(browser,querystr):
    
    text=querystr.strip()
    text=urllib.quote_plus(text)
    
    queryurl='https://cnxxx.awmdm.com/AirWatch/Device/List/Search?Page=&Size=100&SearchText='+text
    
    #print queryurl
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Referer']='https://awmdm.com/AirWatch/'
    
    browser.open(queryurl)
    #print browser.response.content
    
    #     try:
    
    result_trs= browser.find("div",{"id": "DeviceGrid"}).findAll('tr',{'data-action-names':"QueryAll LockDevice SendMessage ChangeOrganizationGroup DeleteDevice"})
    if len(result_trs)>0:
        print "Number of devices found: "+str(len(result_trs))
    else:
        sys.exit('Number of devices found: 0')
    
    for r in result_trs:
        device=idevice()
        
        tr=BeautifulSoup(str(r),'lxml')
        tags=tr.findAll('td')
        for t in tags:     #if t.has_attr('data-property') and t.attrs['data-property']== 'SerialNumber':
            if t.has_attr('data-property'):
                if t.attrs['data-property'] in device.getAttrs():
                    if t.attrs['data-property']=='FriendlyName':
                        device.attrs['FriendlyName']=(t.find('a').text)
                    else:
                        device.attrs[t.attrs['data-property']]=t.text
        
        tags=tr.findAll('input')
        for t in tags:
            if t.attrs['type']=='checkbox':
                #print t.attrs['value']
                device.attrs['DeviceId']=t.attrs['value']

#     except:
#         print 'Error on retriving device info...'
#         sys.exit(1)

    if device.attrs['SerialNumber'] !=querystr:
        sys.exit('Returned device s/n does not match the one that was searched...')
    
    return device

def createBrowser(user,passwd):
    
    
    browser = RoboBrowser(history=True)
    
    browser.open('https://cnxxx.awmdm.com/AirWatch/Login/Login/Login-User')
    
    loginForm = browser.get_form()
    loginForm['UserName']=user
    loginForm['Password']=passwd
    
    browser.submit_form(loginForm)
    resp = json.load(StringIO(browser.response.content))
    
    _success=resp['IsSuccess']
    
    if _success == False:
        print "Login failed...."
        sys.exit(1)
    
    return browser

#def rename(browser,schoolcode,device):
def rename(browser,schoolcode,device_sn,device_id):    
    newname='A'+str(schoolcode)+device_sn
    
    #browser.open('https://cnxxx.awmdm.com/AirWatch') 
    
    url='https://cnxxx.awmdm.com/AirWatch/Devices/DeviceDetails.EditDevice/'+str(device_id)
     
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    #browser.session.headers['hash']='#/Devices/DeviceDetails.EditDevice/450182'
    browser.session.headers['Referer']='https://cnxxx.awmdm.com/AirWatch'
    
    browser.open(url)
    
    #response=browser.response.content
    #print response
    form=browser.get_form()
    
    form['FriendlyName']=newname
    browser.submit_form(form)
    resp = json.load(StringIO(browser.response.content))
    
    #print resp
    _success=resp['IsSuccess']
    
    if _success == False:
        print "Changing name failed...."
    else:
        print "Changing name is successful..."

def findToken(browser):
    
    data  = browser.find_all("script")
    paragraphs = []
    
    for x in data:
        paragraphs.append(str(x)) 
     
    regex='(?<=value=&quot;).{151}'
    matches = re.finditer(regex, " ".join(str(x) for x in paragraphs)) 
    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1
 
        return match.group()
 

def installKhan(browser,device_id,appid):
    app_id=appid
    app_url='https://cnxxx.awmdm.com/AirWatch/#/AirWatch/Device/Details/Apps/'+str(device_id)
    app_url_install='https://cnxxx.awmdm.com/AirWatch/Devices/InstallApp'
    app_url_confirm='https://cnxxx.awmdm.com/AirWatch/Devices/InstallApp?deviceId='+str(device_id)+'&appId='+app_id+'&deviceTypeId=2&source=2'
 
    browser.open(app_url)
    
    token=findToken(browser)
    
    data={'deviceId':device_id,'__RequestVerificationToken':token,'deviceTypeId':'2','selectedPrimaryKeys':app_id+'|org.|False'}
          
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    #browser.session.headers['hash']='#/Devices/DeviceDetails.EditDevice/450182'
    browser.session.headers['Referer']='https://cnxxx.awmdm.com/AirWatch'
     
    #browser.session.post(app_url_install,token)
    browser.session.post(app_url_install,data)
     
    browser.open(app_url_confirm)
    
    #print browser.response.content
    
    try:
        resp = json.load(StringIO(browser.response.content))
    except:
        print device_id 
        print browser.response.content
        
    _success=resp['IsSuccess']

    if _success == True:
        print "Installing app is successful...." 
    else: 
        print "Installing app failed...." 
        
def requestOsUpdate(browser,device_id):
    update_url='https://cnxxx.awmdm.com/AirWatch/Devices/RequestOsUpdates/'
    token=findToken(browser)
    browser.session.post(update_url+device_id,token)
    #print browser.response.content
    
def setRole(role_id,browser):
    print 'setting OU to '+role_id
    RoleCookies={
    'ic4DUh/NXVp8VOKAtyDgbA==':'',
    'MVHG2D97GH8n5jFRanbzhQ==':'',
    '0od0Xt3gw7UG3qUZKSaZvA==':''
    }
    
    browser.open('https://cnxxx.awmdm.com/AirWatch/')
    token=findToken(browser)
    
    url='https://cnxxx.awmdm.com/AirWatch/Users/set-role'
    data={'id':role_id,'__RequestVerificationToken':token}
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Accept']='*/*' 
    browser.session.headers['Host']='cnxxx.awmdm.com'
    browser.session.headers['Origin']='https://cnxxx.awmdm.com'
    browser.session.headers['Pragma']='no-cache'
    browser.session.headers['Referer']='https://cnxxx.awmdm.com/AirWatch/'
    browser.session.headers['Connection']='keep-alive' 
    browser.session.headers['Content-Type']='application/x-www-form-urlencoded; charset=UTF-8'
                            
    #cookies_old=browser.session.cookies.get_dict()
    oldRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if oldRole in RoleCookies.keys():
        print 'Old Role: '+ str(RoleCookies[oldRole])
     
    
    browser.session.post('https://cnxxx.awmdm.com/AirWatch/Users/set-role',data)
    
    newRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if newRole in RoleCookies.keys():
        print 'Set new role to :'+str(RoleCookies[newRole])
          
            

def main():
    schoolcode=raw_input("school code:")
    sns=[]
    appid='9423'
    #appid='71318'
    
    schoolcode=schoolcode.strip()
    # user = raw_input("Username:")
    # passwd = getpass.getpass("Password for " + user + ":")
    user=''
    passwd=''
    
    browser=createBrowser(user,passwd)
    
    #setRole(Roles.Elementary.value,browser)
    
    count=0
    err_devices=[]
     
    devices = searchAll(browser,schoolcode) 
    
    #print devices
    global deviceIds
    deviceIds=[]
    count=0
    for k in devices:
        
        try:
            count+=1
            print 'Renaming number '+str(count)+' of '+str(len(devices))
            #device=queryOne(browser,k.strip())
            time.sleep(2)
            #print 'Renaming '+device.attrs['FriendlyName'] +' to A'+str(schoolcode)+device.attrs['SerialNumber']
            #rename(browser,schoolcode,device)
            rename(browser,schoolcode,k,devices[k])
            deviceIds.append(devices[k])
            
            #time.sleep(2)
            
#             requestOsUpdate(browser,k) 
#             time.sleep(5)
            #installKhan(browser,devices[k])
             
        except:
            err_device.append(k)
            
    #time.sleep(60)
#     for k in deviceIds:
#         installKhan(browser,k,appid)
        
#     time.sleep(10)
    
#     for k in deviceIds:
#         installKhan(browser,k,appid)
    
    print 'Proccessed '+str(count)+' devices'
    if len(err_devices)>0:
        print 'Devices with errors:'
        for d in err_devices:
            print d
            
    print  "Finished....."

if __name__ == '__main__':
    main()



In [43]:
devices

NameError: name 'devices' is not defined

In [28]:
###
# pushing twice khan 
###


    
    
def installKhan(browser,device_id,appid):
    app_id=appid
    app_url='https://cnxxx.awmdm.com/AirWatch/#/AirWatch/Device/Details/Apps/'+str(device_id)
    #app_url_install='https://cnxxx.awmdm.com/AirWatch/Devices/InstallApp?deviceId='+str(device_id)+'&deviceTypeId=2&selectedPrimaryKeys=7053|org.khanacademy.Khan-Academy|False'
    app_url_install='https://cnxxx.awmdm.com/AirWatch/Devices/InstallApp'
    app_url_confirm='https://cnxxx.awmdm.com/AirWatch/Devices/InstallApp?deviceId='+str(device_id)+'&appId='+app_id+'&deviceTypeId=2&source=2'
 
    browser.open(app_url)
    
    token=findToken(browser)
    
    data={'deviceId':device_id,'__RequestVerificationToken':token,'deviceTypeId':'2','selectedPrimaryKeys':app_id+'|org.khanacademy.Khan-Academy|False'}
          
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    #browser.session.headers['hash']='#/Devices/DeviceDetails.EditDevice/450182'
    browser.session.headers['Referer']='https://cnxxx.awmdm.com/AirWatch'
     
    #browser.session.post(app_url_install,token)
    browser.session.post(app_url_install,data)
     
    browser.open(app_url_confirm)
    
    #print browser.response.content
    
    try:
        resp = json.load(StringIO(browser.response.content))
    except:
        print device_id 
        print browser.response.content
        
    _success=resp['IsSuccess']

    if _success == True:
        print "Installing Khan is successful...." 
    else: 
        print "Installing Khan failed...." 
        
user=''
passwd=''

browser=createBrowser(user,passwd)
appid='10355'
#setRole(Roles.Elementary.value,browser) 
for k in deviceIds:
    installKhan(browser,k,appid)
        
time.sleep(10)

for k in deviceIds:
    installKhan(browser,k,appid)    
    
print 'Finished...'

In [6]:
# Move OU and move back

import pip 
import csv,sys,json,os,re
import urllib 
import time

try:
    from robobrowser import RoboBrowser
except :
    pip.main(['install', 'robobrowser'])
from StringIO import StringIO
import getpass,time
#import threading

try:
    from enum import Enum
except :
    pip.main(['install', 'enum'])
try:
    import lxml
except :
    pip.main(['install', 'lxml'])

try:
    from bs4 import BeautifulSoup
except :
    pip.main(['install', 'bs4'])
    
    
    
class Roles(Enum):
    Elementary = '18536'
    Secondary = '18985'
    TOP_LEVEL= '18535'
     
class OUs(Enum):
    ios = 'ios'
    tv = 'tv'
    ioslease= 'ioslease'
    elementary='2717' 
    secondary='2722'
    
def chunks(data, SIZE=10):
    it = iter(data)
    for i in xrange(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
######################################################################
# Modified search method to query by school code for leased iPads
######################################################################

def searchAll(browser,schoolcode):
    devices={}
    text=schoolcode
    text=urllib.quote_plus(text)
    
    queryurl='https://cnxxx.awmdm.com/AirWatch/Device/List/Search?Page=&Size=10&SearchText='+text+'&CurrentView=Custom&Order=Descending&Sort=LastPingDate'
 
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Referer']='https://cnxxx.awmdm.com/AirWatch/'

   
    browser.open(queryurl)
      
    #print browser.response.content
     
    pattern = re.compile("^[A,a]"+schoolcode.strip()+"[a-zA-Z0-9]{12}$") 

    try:
                 
        result_trs= browser.find("div",{"id": "DeviceGrid"}).findAll('tr',{'data-action-names':"QueryAll LockDevice SendMessage ChangeOrganizationGroup DeleteDevice"})
          
        if len(result_trs)>0:
            print "Number of devices found: "+str(len(result_trs)) 
        else:
            print "Number of devices found: 0"
            sys.exit('Exiting....')
            
        for r in result_trs: 
            dev_id=None
            dev_sn=None
            tr=BeautifulSoup(str(r),'lxml')
            tags_td=tr.findAll('td')
            
            tags_input=tr.findAll('input')
            for t in tags_input:
                if t.attrs['type']=='checkbox':
                    #print t.attrs['value']
                    dev_id=t.attrs['value']
            ###
            ### find device that matches new naming convention
            ###
            
            for t in tags_td:    
                if t.has_attr('data-property') and t.attrs['data-property']== 'FriendlyName':
                    if len(t.text.strip().split())<2:
                        if pattern.match(t.text.split()[0]) != None: 
                            print t.text.split()[0]

                            dev_sn=t.text.split()[0].strip()[5:]
                            #print dev_sn
                            #sys.exit(1)
#                         if t.has_attr('data-property') and t.attrs['data-property']== 'SerialNumber':
#                             print t.text.strip()
                        else: continue     
            if dev_sn !=None:
                devices[dev_sn]=dev_id 
    except:
        print 'Error on finding serial number...'
        sys.exit()
    
    #src = str(browser.parsed())

    #result = re.search('%s(.*)%s' % (start, end), src).group(1)
    #print devices  
#     if os.path.exists('sns.json'):
#         os.remove('sns.json')
#     with open('sns.json', 'w') as file:
#          file.write(json.dumps(devices))
    if len(devices)==0:
         sys.exit('Didn\'t find any devices matching naming convention')
    return devices

       
# MoveOU and move back
def moveOU(browser,devices,schoolcode,group_type):
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
 
    if group_type==None or schoolcode==None:
        sys.exit('Error on moving to group: school code or device_type')
    lst=[]
    for g in chunks(devices):
        lst.append(g)
     
    grp='[{""}]
    str_json=json.loads(grp)
    
    ioslease=None
    tv=None
    ios=None e 
 
    for s in str_json:
        if schoolcode in s['label']:
            #print s['label'].lower()[-3:]
            if 'ioslease' in s['label'].lower(): 
                ioslease= str(s['LocationGroupId']) 
            if 'ios' == s['label'].lower()[-3:]:
                ios=str(s['LocationGroupId'])
            if 'appletv' in s['label'].lower():
                tv=str(s['LocationGroupId']) 
        
                
    for g in lst:
        changeurl1='https://cnxxx.awmdm.com/AirWatch/Devices/ChangeDeviceLocationGroupMulti?selectedPrimaryKeys='+','.join(g.values())    
        #changeurl2='https://cnxxxx.awmdm.com/AirWatch/Devices/ChangeDeviceLocationGroupMulti'
         
        browser.open(changeurl1)
        form=browser.get_form()
         
        if group_type=='ios':
            form['NewLocationGroupId']=ios
            print 'moving to '+str(ios)+' group'
            
        if group_type=='tv':
            form['NewLocationGroupId']=tv
            print 'moving to ios '+str(tv)+' group'
            
        if group_type=='ioslease':
            form['NewLocationGroupId']=ioslease
            print 'moving to '+str(ioslease)+' group'
            
        if group_type==OUs..name:
            print 'moving to '+OUs..name+' group'
            form['NewLocationGroupId']=OUs..value
            
        if group_type==OUs.elementary.name:
            print 'moving to '+OUs.elementary.name+' group'
            form['NewLocationGroupId']=OUs.elementary.value
            
        if group_type==OUs.secondary.name:
            print 'moving to '+OUs.secondary.name+' group'
            form['NewLocationGroupId']=OUs.secondary.value
          
        browser.submit_form(form)
        resp = json.load(StringIO(browser.response.content))

        #print resp
        _success=resp['IsSuccess']

        if _success == False:
            print "Changing group failed...."
            sys.exit(resp)
        else: 
            print "Changing group is successful on "+str(len(g))+" devices: "+str(g.keys())

def createBrowser(user,passwd):
    
    
    browser = RoboBrowser(history=True)
    
    browser.open('https://awmdm.com/AirWatch/Login/Login/Login-User')
    
    loginForm = browser.get_form()
    loginForm['UserName']=user
    loginForm['Password']=passwd
    
    browser.submit_form(loginForm)
    resp = json.load(StringIO(browser.response.content))
    
    _success=resp['IsSuccess']
    
    if _success == False:
        print "Login failed...."
        sys.exit(1)
    
    return browser


def setRole(role_id,browser):
    print 'setting OU to '+role_id
    RoleCookies={
    'ic4DUh/NXVp8VOKAtyDgbA==':'Elementary_Top_Level',
    'MVHG2D97GH8n5jFRanbzhQ==':'Top_Level',
    '0od0Xt3gw7UG3qUZKSaZvA==':'Secondary_Top_Level'
    }
    
    browser.open('https://awmdm.com/AirWatch/')
    token=findToken(browser)
    
    url='https://awmdm.com/AirWatch/Users/set-role'
    data={'id':role_id,'__RequestVerificationToken':token}
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Accept']='*/*' 
    browser.session.headers['Host']='awmdm.com'
    browser.session.headers['Origin']='https://awmdm.com'
    browser.session.headers['Pragma']='no-cache'
    browser.session.headers['Referer']='https://awmdm.com/AirWatch/'
    browser.session.headers['Connection']='keep-alive' 
    browser.session.headers['Content-Type']='application/x-www-form-urlencoded; charset=UTF-8'
                            
    #cookies_old=browser.session.cookies.get_dict()
    oldRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if oldRole in RoleCookies.keys():
        print 'Old Role: '+ str(RoleCookies[oldRole])
     
    
    #browser.open('https://awmdm.com/AirWatch/Users/set-role?id='+role_id+'&__RequestVerificationToken='+token,method='post')
    browser.session.post('https://awmdm.com/AirWatch/Users/set-role',data)
    
    newRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if newRole in RoleCookies.keys():
        print 'Set new role to :'+str(RoleCookies[newRole])
        
def findToken(browser):
    
    data  = browser.find_all("script")
    paragraphs = []
    
    for x in data:
        paragraphs.append(str(x)) 
     
    regex='(?<=value=&quot;).{151}'
    matches = re.finditer(regex, " ".join(str(x) for x in paragraphs)) 
    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
        #print match.group()
        return match.group()
    
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1

#             print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))


        
            

def main():
    
#     with open('sns.json', 'r') as f:
#         try:
#             devices = json.load(f)
#         # if the file is empty the ValueError will be thrown
#         except ValueError:
#             print 'Importing sns.json error'
#             devices = {}

    
    schoolcode=raw_input("school code:")
    sns=[]
    schoolcode=schoolcode.strip()
    
  
    # user = raw_input("Username:")
    # passwd = getpass.getpass("Password for " + user + ":")
    user=''
    passwd=''
    
    browser=createBrowser(user,passwd) 
    
    #setRole(Roles.Elementary.value,browser)
    
    count=0
    err_devices=[] 
    setRole(Roles.Elementary.value,browser)
    devices=searchAll(browser,schoolcode)
     
    print 'Found '+str(len(devices))+' matches'
    print '\n'
    print 'Moving devices out of OU' 
#     try: 
    moveOU(browser,devices,schoolcode,OUs.elementary.name)   
#     count+=1
#     except:
#             err_devices.append(k)
           
#     print 'Proccessed '+str(count)+' devices'
#     if len(err_devices)>0:
#         print 'Devices with errors'
#         for d in err_devices:
#             print d
    print 'Waiting for 1 minute...'     
    time.sleep(60)
    print 'Moving devices back into original OU'
#     try: 
    moveOU(browser,devices,schoolcode,OUs.ioslease.name)   
#     count+=1
#     except:
#             err_devices.append(k)
            
    
#     print 'Proccessed '+str(count)+' devices'
#     if len(err_devices)>0:
#         print 'Devices with errors'
#         for d in err_devices:
#             print d
                   
    print  "Finished....."
    

if __name__ == '__main__':
    main() 

school code:0344
setting OU to 18536
Old Role: Elementary_Top_Level
Set new role to :Elementary_Top_Level
Number of devices found: 21


SystemExit: Didn't find any devices matching naming convention

To exit: use 'exit', 'quit', or Ctrl-D.


In [103]:
pattern = re.compile("^[A,a]\d{4}[a-zA-Z0-9]{12}$") 
string ='A0276DMPTWCXYHLF9'
if pattern.match(string)!=None:
    print 'not none' 
else: 
    print 'none'

not none


In [16]:
s=''
print s.split()[0]
code=''
pattern = re.compile("^[A,a]"+schoolcode.strip()+"[a-zA-Z0-9]{12}$") 
type(pattern.match(s.split()[0]))


In [61]:
len(deviceIds)